# A template-based table-maker

This document proposes a template-based design to render regression tables and similar parameter-presentations.

## Naming

The most important part of new package, the name.

- `import tabula_rasa as tr`

## Problems with existing solutions



## Proposal

In [23]:
import jinja2
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

from IPython.core.display import HTML
from pathlib import Path
from estimagic_stargazer.stargazer import extract_info_from_sm
from estimagic_stargazer.stargazer import extract_params_from_sm

In [24]:
def generate_random_data():
    np.random.seed(1)

    n_samples = 10_000

    constant = 0.1
    epsilon = np.random.normal(size=n_samples)
    beta = np.random.uniform(-1, 1)

    df = pd.DataFrame({"x": np.random.normal(loc=0, scale=1, size=n_samples),})

    z = constant + beta * df.x + epsilon
    pr = 1 / (1 + np.exp(-z))

    df["y"] = np.random.binomial(1, pr)
    
    return df


def generate_params_and_info(df):
    model = smf.ols(formula="y ~ x", data=df).fit()
    model_info = {
        "params": extract_params_from_sm(model),
        "info": extract_info_from_sm(model),
    }
    # Add number of observations.
    model_info["info"]["n_obs"] = 10_000   
    
    return model_info

## Examples

In the following, I am going to demonstrate the capabilities of templates using, first, an OLS regression and, secondly, I show how to make a Logit regression table without changing the source code, just by allowing the user to overwrite templates. We start with a `params` DataFrame and a `namedtuple` with additional information.

### Example 1 - Syntax and Footer

The first example shows a little bit of Jinja Syntax.

In [25]:
df = generate_random_data()
model = generate_params_and_info(df)
model['params']

,value,pvalue,standard_error,ci_lower,ci_upper
Intercept,0.523305,0.000000e+00,0.004779,0.513937,0.532673
x,0.143708,5.500295e-194,0.004730,0.134436,0.152981


Let us start with a simple Jinja2 template for the footer and play a little bit around.

The following cell contains the code for the footer. The section does not include any borderlines or other sections like the parameters and notes. The combination of the individual pieces will be handled by a master document later.

There are two different Jinja syntaxes in this template.

- Double curly braces like `{{ n_obs }}` allow to insert Python variables into the template.
- `{% if n_obs is defined %}<do something>{% endif %}` are if-elif-else constructs. There is a special check for whether a variable is supplied to the template or not. This allows to omit variables.

In [26]:
footer = """
<table style="text-align:center">
    </tr>
    {% if n_obs is defined %}
    <tr>
        <td style="text-align: left">Observations</td>
        <td>{{ n_obs }}</td>
    </tr>
    {% endif %}
    {% if rsquared is defined %}
    <tr>
        <td style="text-align: left">R<sup>2</sup></td>
        <td>{{ rsquared }}</td>
    </tr>
    {% endif %}
    {% if rsquared_adj is defined %}
    <tr>
        <td style="text-align: left">Adjusted R<sup>2</sup></td>
        <td>{{ rsquared_adj }}</td>
    </tr>
    {% endif %}
    {% if fvalue is defined %}
    <tr>
        <td style="text-align: left">F Statistic</td>
        <td>{{ fvalue }}<sup>****</sup>{% if df_model is defined and df_resid is defined %}(df = {{ df_model }}; {{ df_resid }}){% endif %}</td>
    </tr>
    {% endif %}
</table>
"""
HTML(footer)

Observations,{{ n_obs }}
R2,{{ rsquared }}
Adjusted R2,{{ rsquared_adj }}
F Statistic,{{ fvalue }}****{% if df_model is defined and df_resid is defined %}(df = {{ df_model }}; {{ df_resid }}){% endif %}


Here we turn the string into a template.

In [27]:
template = jinja2.Template(footer)

We can supply variable to the template and render it.

In [28]:
HTML(template.render(n_obs=100))

Observations,100


In [30]:
HTML(template.render(rsquared=0.95, n_obs=100,rsquared_adj=0.9))

Observations,100
R2,0.95
Adjusted R2,0.9


In [32]:
HTML(template.render(fvalue=0.9, n_obs=100,df_model=1))

Observations,100
F Statistic,0.9****


In [33]:
HTML(template.render(fvalue=0.9, df_model=1,n_obs=100, df_resid=999))

Observations,100
F Statistic,0.9****(df = 1; 999)


### Example 2 - Environments and Master Document

We take another step. Here we are going to define the master document which embeds the footer and look at template environments which are collections of templates.

First, we are going to write the templates for the master document and the footer to the disk and load the templates into a template environment. The new directive to learn is `{% extends 'asd' %}` which indicates that this template allows to extend the mentioned template. Extensions are defined as blocks. We wrap the whole footer into a block called footer.

The Jinja2 feature is called [template inheritance](https://jinja.palletsprojects.com/en/2.11.x/templates/#template-inheritance) and seems unnecessary complicated at first, but it will allow the user to inject templates herself and overwrite existing ones.

In [10]:
path = Path("templates").resolve()
path.mkdir(exist_ok=True)
print(path)

C:\Users\admin\Documents\ra\estimagic_stargazer\templates


In [54]:
%%writefile templates/ols.html
<table style="text-align:center">

    <tr>
        <td colspan="2" style="border-bottom: 1px solid black">{{ title }}</td>
    </tr>

    <td colspan="2" style="border-bottom: 1px solid black"></td>

    {% block footer %}Default block value. Should be overwritten by footer.html. WHY?!{% endblock %}

    <td colspan="2" style="border-bottom: 1px solid black"></td>

</table>

Overwriting templates/ols.html


In [55]:
%%writefile templates/footer.html
{% extends "ols.html" %}

{% block footer %}
    Hello
{% if n_obs is defined %}
<tr>
    <td style="text-align: left">Observations</td>
    <td>{{ n_obs }}</td>
</tr>
{% endif %}
{% if rsquared is defined %}
<tr>
    <td style="text-align: left">R<sup>2</sup></td>
    <td>{{ rsquared }}</td>
</tr>
{% endif %}
{% if rsquared_adj is defined %}
<tr>
    <td style="text-align: left">Adjusted R<sup>2</sup></td>
    <td>{{ rsquared_adj }}</td>
</tr>
{% endif %}
{% if fvalue is defined %}
<tr>
    <td style="text-align: left">F Statistic</td>
    <td>{{ fvalue }}<sup>****</sup>{% if df_model is defined and df_resid is defined %}(df = {{ df_model }}; {{ df_resid }}){% endif %}</td>
</tr>
{% endif %}
{% endblock %}

Overwriting templates/footer.html


Next, here comes the master document. At an appropriate position, we insert the block directive which will be filled with the footer.

Now, we collect the templates in an [environment](https://jinja.palletsprojects.com/en/2.11.x/api/#jinja2.Environment). You require the environment and loaders to collect the templates. [Loaders](https://jinja.palletsprojects.com/en/2.11.x/api/#loaders) exist in various forms.

In [60]:
loader = jinja2.FileSystemLoader(path)
env = jinja2.Environment(
    autoescape=jinja2.select_autoescape(['html', 'htm', 'xml']),
    loader=loader,
)

Let us render the master document.

In [68]:
template = env.get_template("footer.html")

In [69]:

HTML(template.render(title="OLS Regression", n_obs=1000, fvalue=0.9))

# Janos has an idea

DataFrame to html and latex and give the user option to call to_latex